In [1]:
import pandas as pd

a = pd.read_csv("/home/changly/11695/group-project-s23-moviegpt/src/data_process/data_files/train_data.csv")
a.shape

(127719, 18)

In [3]:
import pickle
import os
# absolute_path = os.path.dirname(__file__)
# cr_path = "content_recommend"
with open("content_recommend/base_recommend.pkl", "rb") as f:
    base_recommend = pickle.load(file=f)
base_recommend

['dragonfly+1976',
 'reckless+1995',
 'vessel+of+wrath+1938',
 'blessed+event+1932',
 'chilly+scenes+of+winter+1979',
 'the+misadventures+of+margaret+1998',
 'heavy+weather+1995',
 'the+crush+1967',
 'the+great+kidnapping+1973',
 'vessel+of+wrath+1938',
 'little+dorrit+1987',
 'canned+dreams+2012',
 'canned+dreams+2012',
 'children+in+the+wind+1937',
 'as+i+was+moving+ahead+occasionally+i+saw+brief+glimpses+of+beauty+2000',
 'dilwale+dulhania+le+jayenge+1995',
 'love+torn+in+a+dream+2000',
 'serving+life+2011',
 'deadline+2004',
 'position+among+the+stars+2011']

In [2]:
a.columns

Index(['user_id', 'title_year', 'user_rating', 'tmdb_id', 'title', 'overview',
       'genres', 'cast', 'crew', 'keywords', 'popularity', 'release_date',
       'runtime', 'vote_average', 'vote_count', 'age', 'occupation', 'gender'],
      dtype='object')

In [8]:
a['overview'][0]

"As Harry begins his sixth year at Hogwarts, he discovers an old book marked as 'Property of the Half-Blood Prince', and begins to learn more about Lord Voldemort's dark past."

In [3]:
from content_based_filteringv2 import UserDataLoader
user_data_loader = UserDataLoader(a)
user_data_loader.get_user_feats()

In [4]:
from content_based_filteringv2 import ContentFiltering
import warnings
warnings.filterwarnings("ignore")
train_data = a[['user_id', 'title_year', 'user_rating', 'age', 'gender', 'occupation', 'overview']]
content_model = ContentFiltering()
content_model.train(train_data)

In [13]:
train_data = a[['user_id', 'title_year', 'user_rating', 'age', 'gender', 'occupation', 'overview']]
train_data['title_year_overview'] = train_data["title_year"] + " " + train_data["overview"]
train_data['title_year_overview'] = train_data['title_year_overview'].str.replace('+', ' ')
train_data = train_data[["title_year", "overview"]].drop_duplicates(ignore_index=True)
train_data.shape

/tmp/ipykernel_8249/11990642.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['title_year_overview'] = train_data["title_year"] + " " + train_data["overview"]
/tmp/ipykernel_8249/11990642.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train_data['title_year_overview'] = train_data['title_year_overview'].str.replace('+', ' ')
/tmp/ipykernel_8249/11990642.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

(19299, 2)

In [9]:
train_data.columns

Index(['title_year', 'overview'], dtype='object')

In [12]:
train_data = a[['user_id', 'title_year', 'user_rating', 'age', 'gender', 'occupation', 'overview']]
train_data['title_year_overview'] = train_data["title_year"] + " " + train_data["overview"]
train_data['title_year_overview'] = train_data['title_year_overview'].str.replace('+', ' ')
train_data_overview = train_data[["title_year_overview", "title_year"]].drop_duplicates(subset=['title_year'], ignore_index=True)
train_data_overview.shape

/tmp/ipykernel_8249/2947136836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['title_year_overview'] = train_data["title_year"] + " " + train_data["overview"]
/tmp/ipykernel_8249/2947136836.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train_data['title_year_overview'] = train_data['title_year_overview'].str.replace('+', ' ')
/tmp/ipykernel_8249/2947136836.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

(19299, 2)

In [8]:
train_data.head(5)

,user_id,title_year,user_rating,age,gender,occupation,overview
0,545008,harry+potter+and+the+half-blood+prince+2009,2,19.0,M,scientist,"As Harry begins his sixth year at Hogwarts, he..."
1,323296,leon+the+professional+1994,4,28.0,M,college/grad student,"Leon, the top hit man in New York, has earned ..."
2,426278,armageddon+1998,4,28.0,M,scientist,When an asteroid threatens to collide with Ear...
3,358548,the+parallax+view+1974,4,34.0,M,executive/managerial,An ambitious reporter gets in way-over-his-hea...
4,64177,x-men+days+of+future+past+2014,5,32.0,M,executive/managerial,The ultimate X-Men ensemble fights a war for t...


In [11]:
train_data_overview = train_data[["overview"]].drop_duplicates(ignore_index=True)
tfidf = content_model.tfidf_vectorizer.fit_transform(train_data_overview["overview"].astype('U').values)
tfidf.shape

(19003, 491685)

In [13]:
train_data_overview.shape

(19003, 1)